# 981. Time Based Key-Value Store

### Difficulty: <font color = orange> Medium </font>

---

Design a time-based key-value data structure that can store multiple values for the same key at different time stamps and retrieve the key's value at a certain timestamp.

Implement the TimeMap class:

- `TimeMap()` Initializes the object of the data structure.

- `void set(String key, String value, int timestamp)` Stores the key `key` with the value `value` at the given time `timestamp`.

- `String get(String key, int timestamp)` Returns a value such that `set` was called previously, with `timestamp_prev <= timestamp`. If there are multiple such values, it returns the value associated with the largest `timestamp_prev`. If there are no values, it returns `""`.


---

Example 1:

Input

`["TimeMap", "set", "get", "get", "set", "get", "get"]`

`[[], ["foo", "bar", 1], ["foo", 1], ["foo", 3], ["foo", "bar2", 4], ["foo", 4], ["foo", 5]]`

Output

`[null, null, "bar", "bar", null, "bar2", "bar2"]`

Explanation

TimeMap timeMap = new TimeMap();

timeMap.set("foo", "bar", 1);  // store the key "foo" and value "bar" along with timestamp = 1.

timeMap.get("foo", 1);         // return "bar"

timeMap.get("foo", 3);         // return "bar", since there is no value corresponding to foo at timestamp 3 and timestamp 2, then the only value is at timestamp 1 is "bar".

timeMap.set("foo", "bar2", 4); // store the key "foo" and value "bar2" along with timestamp = 4.

timeMap.get("foo", 4);         // return "bar2"

timeMap.get("foo", 5);         // return "bar2"
 
----
Constraints:

- `1 <= key.length, value.length <= 100`

- `key` and `value` consist of lowercase English letters and digits.

- `1 <= timestamp <= 107`

- All the timestamps `timestamp` of `set` are strictly increasing.

- At most `2 * 105` calls will be made to `set` and `get`.

## Approach Overview

Use a dictionary to store keys, it's corresponding values and timestamps. 

To add to dictionary (set method): Simply add the keys and it's corresponding values and timestamps and store them in a list (the values and timestamp that is)

To retrieve data from dictionary (get method): We perform a binary search on the list of values associated with the requested key to output the correct value. 


## Detailed Explanation

This one was interesting. It's an OOP problem that require we create a data structure and implement `get` and `set` operations. `set` ops refers to adding dataset to the data structure and `get` ops refers to retrieving data from the data structure.

<b> Defining the data structure </b>

The most appropriate data structure for this problem would be a dictionary. 

Why?

Because we're storing data in a key-value pair format, a dictionary is the most correct choice. The data being stored is also of different types and it's faster & easier to store and retrieve data of different types with a dictionary. 

In our particular case our `key` is of `string` type and `value` of `string and integer` type. 

Our values are the variables `value` and `timestamp` btw and the key is the variable `key`. 

And each `key` can have multiple values, so therefore we use a list of lists to store the values. 

Example: 

`self.store = { key1: [ [value1, timestamp1], [value2, timestamp2], [value3, timestamp3] ] }`

To figure this out we just needed to carefully read and study the problem description.

<b>Adding data to the data structure (dictionary)</b>


Fairly straightforward. Just one line of code. Add a pair of values (value and timestamp) in a list. 

`self.store[key].append([value, timestamp])`

<b> Retrieving data from the data structure (dictionary)</b>

This is a search operation. We are given two inputs. A key and a timestamp and we want to fetch the value corresponding to them. And if this doesn't exists then we fetch the value corresponding to the largest timestamp in the list of stored dataset i.e. `timestamp_prev <= timestamp`

For example say we wanted to fetch the value corresponding to the key `"foo"` at timestamp `4`. But in our `store` dictionary, for the key `"foo"` we don't have a value stored with the timstamp `4`. 

Let's say we only have following list of list of values.

self.store = `{ "foo" : [ ["bar", 1], ["bar1", 2], ["bar2", 3]]}`

The next largest and closest timestamp we have is `3`, and it corresponds to the value `"bar2"`. 

So we therefore return `"bar2"` in this particular case because as per the problem statement we need to look for the next smallest closest timestamp to the input timestamp. i.e.`timestamp_prev <= timestamp`. 

And if the desired timestamp is already in the `store` dictionary, then we just return the value it corresponds to.

<i>Let's talk about the search conditions for a bit as I found it a little tough to wrap my head around</i>


There are only 3 cases we need to account for.


1) The midpoint dataset of the specified `key` (specifically the value of its timestamp) is an exact match with the input timestamp: If true, then simply return it's corresponding value.



2) The midpoint dataset of the specified `key` (specifically the value of its timestamp) is bigger than the input timestamp: If true, then ignore the current dataset and all dataset located to the right of it (because they are bigger than the desired timestamp). And narrow search range to datasets located to the left of current dataset. We do that by decrementing right pointer.
`right = mid - 1`


3) The midpoint dataset of the specified `key` (specifically the value of its timestamp) is smaller than the input timestamp: If true, then don't discard it but save the current value (in the output variable), and THEN narrow the search range by incrementing the left pointer to search for a potentially larger value (located to the right of the current dataset, NOT ON THE LEFT!).

`output = self.store[key][mid][0]` # save the value corresponding to the current `timestamp_prev` in list
`left = mid + 1` # increment left pointer (to look for an even larger value in `timestamp_prev` list)



## Key Challenges

1) Giving up too early and not spending enough time reading carefully through the problem statement. This was a OOP problem. Last I dealt with OOP was at my SWE internship and A Levels, that's some 5 years ago. I got scared and barely spent 2 mins on this before giving up entirely.

2) Implementing the search was a bit hard because I had to operate on a list of lists, alas I found out though that this was not hard at all. I was overthinking it. I think I'm finally starting to get a hang on binary search. The idea is fairly straightforward, initialize the pointers to the first and last element in the input list. Then we calculate the midpoint of the search range and check if the midpoint value is either larger, smaller or equal to the target we're looking for. Depending on the result we then narrow the search range to discard incorrect data range and focus on the correct data range. 

# Solution:

In [ ]:
from collections import defaultdict 
class TimeMap:

    def __init__(self):
    # initialize dictionary to store key-value along with timestamp
        self.store = defaultdict(list)

    def set(self, key: str, value: str, timestamp: int) -> None:
        
        # adding the key's value and timestamp in the dictionary 
        self.store[key].append([value, timestamp])

    def get(self, key: str, timestamp: int) -> str:
        
        # initialize left pointer
        left = 0
        
        # initialize right pointer
        right = len(self.store[key]) - 1
        
        # intializing output with empty string
        output = ""
        
        # continue as long as we don't exceed the bounds of the dictionary's values
        while left <= right:
            
            # calclulate midpoint
            mid = (left + right) // 2
            
            # check if timestamp is already in the key's set of values 
            if self.store[key][mid][1] == timestamp:
                
                # if it is then return the value associated with that timestamp
                return self.store[key][mid][0]
            
            # check if timestamp of the current dataset (midpoint) is larger than the requested timestamp 
            elif self.store[key][mid][1] > timestamp:
                # narrow search range by decrementing right pointer
                right = mid - 1
            
            # check if timestamp of current dataset (midpoint) is smaller than the requested timestamp 
            else:
                # save the value associated with the timestamp 
                output = self.store[key][mid][0]

                # increment left pointer 
                left = mid + 1        
        
        # return output 
        return output